## RFM模型分析

1.重要价值用户 属于企业忠诚用户，按照生命周期看，需要一定的营销策略延长用户忠诚时间，比如VIP服务、永久打折策略等等让用户有荣誉感的营销策略；

2.重要发展用户 近期有发生购买行为，消费金额高于多数用户，但是购买频率较低，需要刺激用户进行复购，增加对品牌的忠诚，可以通过短信、邮件定期发送新品、爆款、折扣等信息，吸引用户增加购买频率；

3.重要保持用户 最近一次消费时间较远，但消费频次和总金额较高，说明是一段时间没有光顾的忠实客户（即将流失），需要我们主动联系进行挽回，可以通过电话沟通，唤起用户对品牌的认知，根据用户反馈制定个性化营销策略，例如发送大额优惠券等；

4.重要挽留用户 历史消费金额较高，但是购买频率较低，近期没有发生购买行为，已经趋于流失，最核心的需求是刺激用户进行复购，增加对产品及品牌的印象，针对这部分用户可以较大的让利，比如推出0元试用、买一送一等服务；

5.一般价值客户 近期购买，购买频率高但是总消费金额低，这部分用户的客单价较低，有薅羊毛的可能性，可以尝试通过站内及站外精准种草，比如抖音等平台信息流投放等推送店铺爆款产品，增加品牌曝光度，提高客单价；

6.一般发展用户 近期有发生购买行为，购买频率和消费金额较低，从用户生命周期上看，处于引入期和成长期，需要通过电话主动联系，关怀产品使用情况，提供良好的售后服务，增加用户对品牌的信赖度，金额提升复购率和消费金额；

7.一般保持用户和一般挽留用户 用户已经处于流失阶段，可以不做营销，或者花费较低成本进行尝试触达

In [ ]:
#RFM model
RFM_index = {
    "高高高":"重要价值客户",
    "高低高":"重要发展客户",
    "低高高":"重要保持客户",
    "低低高":"重要挽留客户",
    "高高低":"一般价值客户",
    "高低低":"一般发展客户",
    "低高低":"一般保持客户",
    "低低低":"一般挽留客户"
}

import pandas as pd

path = 'RFM.xlsx'
data = pd.read_excel(path, sheet_name='Sheet1')

#应用RFM模型只需要以下列
data_use = data[['InvoiceNo', 'Quantity', 'InvoiceDate','UnitPrice', 'CustomerID']]

#column format
def date_format(datetime):
    origin_date = pd.to_datetime(datetime, format="%m/%d/%Y %H:%M")
    format_date = origin_date.strftime("%Y/%m/%d %H:%M")
    return format_date

data_use['InvoiceDate'] = data_use['InvoiceDate'].apply(lambda x:date_format(x))

#计算R值，Recency
customer_r = data_use.groupby(['CustomerID'], as_index=False)['InvoiceDate'].max()
customer_r['R value'] = customer_r['InvoiceDate'].apply( lambda x:(pd.to_datetime("2011/12/31")-pd.to_datetime(x)).days)

#计算F值， Frequency
customer_f = data_use.groupby(['CustomerID'],as_index=False)['InvoiceNo'].nunique()

#计算M值，Monetary
data_use['Cost'] = data_use['Quantity'] * data_use['UnitPrice']
customer_m = data_use.groupby(['CustomerID'],as_index=False)['Cost'].sum()

#迭代合并数据
from functools import reduce
customer_table = [customer_r, customer_f, customer_m]
customer_RFM = reduce(pd.merge, customer_table)

#数据整合
customer_RFM = customer_RFM.drop(['InvoiceDate'], axis=1)
customer_RFM.rename(columns = {'InvoiceNo':'F value', 'Cost':'M value'}, inplace=True)

#根据业务需要和数据特点对RFM进行分级
#暂定用中位数
R_median = customer_RFM['R value'].median()
customer_RFM['R index'] = customer_RFM['R value'].apply(lambda x:'高' if x < R_median else '低')

F_median = customer_RFM['F value'].median()
customer_RFM['F index'] = customer_RFM['F value'].apply(lambda x:'高' if x > F_median else '低')

M_median = customer_RFM['M value'].median()
customer_RFM['M index'] = customer_RFM['M value'].apply(lambda x:'高' if x > M_median else '低')

#map level
customer_RFM['RFM'] = customer_RFM['R index']+customer_RFM['F index']+customer_RFM['M index']
customer_RFM['level'] = customer_RFM['RFM'].apply(lambda x:RFM_index[x])
customer_RFM

In [ ]:
#进一步统计分析
customer_RFM_count = customer_RFM.groupby(['level'],as_index=False)['CustomerID'].nunique()
user_sum = customer_RFM_count['CustomerID'].sum()
customer_RFM_count['user_rate'] = customer_RFM_count['CustomerID'].apply(lambda x: '{:.2%}'.format(x/user_sum))
customer_RFM_count